### Document Translation

In [ ]:
import requests
import os

# Construct the URL for the Document Translation API
endpoint = "https://your-cognitive-service-endpoint.cognitiveservices.azure.com/"
path = "/translator/document:translate"
url = endpoint + path

# Set up the headers with your subscription key
headers = {
    "Ocp-Apim-Subscription-Key": "your_subscription_key"
}

# Define the parameters for the translation request
params = {
    "sourceLanguage": "en",  # Source language code
    "targetLanguage": "zh-Hant",  # Target language code
    "api-version": "2023-11-01-preview"  # API version
}

# Specify the input and output file paths
input_file = "./input_document.docx"  # Path to the input document
output_file = "./output_document.docx"  # Path to save the translated document

# Open the input file in binary mode
with open(input_file, "rb") as document:
    # Define the data to be sent with the request
    data = {
        "document": (os.path.basename(input_file), document, "application/vnd.openxmlformats-officedocument.wordprocessingml.document")
    }

    # Send the POST request to the Document Translation API
    response = requests.post(url, headers=headers, files=data, params=params)

    # Write the translated document to the output file
    with open(output_file, "wb") as output_document:
        output_document.write(response.content)


### Install python-docx

In [ ]:
# Command to install the python-docx library
# pip install python-docx

### Read Word Document


In [ ]:
from docx import Document

# Function to read and extract text from a Word document
def read_word_document(file_path):
    doc = Document(file_path)  # Load the document
    full_text = []
    for paragraph in doc.paragraphs:  # Iterate through paragraphs
        full_text.append(paragraph.text)  # Add paragraph text to list
    return '\n'.join(full_text)  # Join paragraphs into a single string

# Read the translated and original documents
chinese_text = read_word_document('./output_document.docx')
english_text = read_word_document('./input_document.docx')

### Compare Translations

In [ ]:
import requests

# Configuration for the OpenAI API
API_KEY = "your_api_key"
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Payload for the request to compare translations
payload = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant that compares translations."},
        {"role": "user", "content": f"Compare the following texts and check if the English text is an accurate translation of the Chinese text. Chinese: {chinese_text} English: {english_text}"}
    ],
    "temperature": 0.7,  # Temperature setting for response randomness
    "top_p": 0.95,  # Top-p sampling parameter
    "max_tokens": 800  # Maximum tokens in the response
}

# Endpoint for the OpenAI API
ENDPOINT = "https://your-endpoint-url/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

# Send request to the OpenAI API
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad responses
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle and print the response from the API
comparison_result = response.json()
print(comparison_result['choices'][0]['message']['content'])